In [1]:
import data_process, data_functions
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from functools import reduce
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, hamming_loss

In [2]:
#load pmpm data only
#notebooks/causal_data_creation/biometric_matching_X
canary, biometrics, pmpm = data_process.load_data()

In [3]:
canary.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME  \
0  2602482  6F995322100       35938          365 LifePoints   
1  2603072  9H994959900       37330  Medicare Advantage 175   
2  2606370  M0003719100       37330  Medicare Advantage 175   
3  2608221  6J858648100       37330  Medicare Advantage 175   
4  2619378  6K991068900       36550  Medicare Advantage 125   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   
4  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/MOVEI...  2021-01-11   

  COMPLETION_DATE  N_MILESTONES_ACHIEVED LOB   LAST_COMPLETED_MILESTONE  \
0      2019-08-09                      1  DN              REGISTER_DATE   
1      2020-06-29                     17  DN  17TH_LESSON_COMPLETE_DATE   
2      2021-01-06                     12  DN  12TH_LESSON_COMPLETE_DATE   
3      2020-12-30                      8  DN   8TH_LESSON_COMPLETE_DATE   
4      2021-01-08                     20  DN  20TH_LESSON_COMPLETE_DATE   

  REGISTER_DATE LAST_ACTIVITY_DATE  
0    2019-08-09         2019-08-09  
1    2020-01-01         2020-06-29  
2    2020-10-23         2021-01-06  
3    2020-10-12         2020-12-30  
4    2019-10-31         2021-01-08

In [4]:
biometrics.head()

PERS_ID          EVENT_DATE             DATE_TYPE  \
0  2570719 2019-10-03 15:01:27  OBSERVATION_DATETIME   
1  2570719 2018-12-05 00:34:56             CALC_DATE   
2  2570719 2009-08-03 11:13:00  OBSERVATION_DATETIME   
3  2570719 2019-10-15 00:00:00  OBSERVATION_DATETIME   
4  2570719 2019-12-27 00:00:00  OBSERVATION_DATETIME   

           BIOMETRIC_CATEGORY EVENT_CODE CODING_STANDARD      SOURCE  \
0                      Height     8302-2                       HIXNY   
1                         BMI       None            None  CALCULATED   
2  Blood Pressure (Diastolic)     8462-4              LN       HIXNY   
3                      Height     8302-2              LN       HIXNY   
4                      Weight     3141-9              LN       HIXNY   

         RESULT_VALUE  
0    70.0000390014801  
1  26.398623434221204  
2                60.0  
3                70.0  
4  178.39999389648438

In [5]:
pmpm.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570122               95      40392        0             0   
1  2570122               94      19837        0             0   
2  2570268               93       4922        0             1   
3  2570268               94        232        0             1   
4  2570268               93       2776        0             1   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  1       0          1                    0             0   
1                  0       0          1                    0             1   
2                  1       0          1                    0             1   
3                  1       0          1                    0             1   
4                  1       0          1                    0             1   

   ...  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  \
0  ...                 0                       1               0   
1  ...                 0                       0               0   
2  ...                 0                       0               0   
3  ...                 0                       0               0   
4  ...                 0                       0               0   

   CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  \
0                0          0           0                   0   
1                0          0           0                   0   
2                0          0           0                   0   
3                0          0           0                   0   
4                0          0           0                   0   

   CC_VISUAL_IMPAIR  MBR_GNDR     MYR  
0                 0         M  202002  
1                 0         M  201810  
2                 0         F  201704  
3                 0         F  201807  
4                 0         F  201801  

[5 rows x 74 columns]

# Functions

In [6]:
def process_biometrics(biometrics):
    biometrics.EVENT_DATE = biometrics.EVENT_DATE.apply(lambda x: x.strftime("%Y%m"))
    biometrics = biometrics[["PERS_ID", "EVENT_DATE", "BIOMETRIC_CATEGORY", "RESULT_VALUE"]]
    biometrics = biometrics.dropna()
    biometrics = biometrics.loc[biometrics.PERS_ID>'0']
    biometrics.RESULT_VALUE = biometrics.RESULT_VALUE.astype('float')
    biometrics = biometrics.set_index(['PERS_ID', 'EVENT_DATE']).sort_index()
    biometrics = pd.pivot_table(data=biometrics, index=biometrics.index, values='RESULT_VALUE', columns='BIOMETRIC_CATEGORY', aggfunc='mean').fillna(0)
    biometrics = biometrics.reset_index()
    biometrics[['PERS_ID', 'MYR']] = pd.DataFrame(biometrics['index'].tolist(), index=biometrics.index)
    biometrics = biometrics.drop(['index'],axis=1)
    cols = ["PERS_ID"] + ["MYR"] + biometrics.columns.tolist()[:-2]
    biometrics = biometrics.reindex(columns=cols)
    return biometrics

In [7]:
def process_PMPM(pmpm, start_MYR, end_MYR, top_diseases, show_range = False):
    
    old_cols = pmpm.columns.to_list()
    new_cols = old_cols[:2] + list([old_cols[-2]]) + list([old_cols[-1]]) + old_cols[2:-2]
    pmpm = pmpm.reindex(columns=new_cols)
    
    sorted_cols = pmpm.sum(axis=0, numeric_only=True).sort_values(ascending=False).head(top_diseases+4)
    disease_cols = sorted_cols.index.to_list()[4:]
    disease_cols.append("CC_DIABETES") #special
    
    new_pmpm = pmpm.loc[(pmpm.MYR>=start_MYR)&(pmpm.MYR<=end_MYR)&(pmpm.PERS_ID>'0')].sort_values(by=["PERS_ID","MYR"]).reset_index(drop=True)
    data = pd.pivot_table(data=new_pmpm,index="PERS_ID",values=disease_cols, aggfunc="sum")
    data[data>=1] = 1
    
    if show_range==True:
        data.insert(0,"MIN_MYR", new_pmpm[['PERS_ID', 'MYR']].groupby('PERS_ID').agg('min'),allow_duplicates=False)
        data.insert(1,"MAX_MYR", new_pmpm[['PERS_ID', 'MYR']].groupby('PERS_ID').agg('max'),allow_duplicates=False)
        data.insert(2,"MONTHS_MYR", data.MAX_MYR - data.MIN_MYR + 1, allow_duplicates=False)
        
    data = data.reset_index()
    
    return data
        

In [8]:
def generate_common_patients(pmpm_X, pmpm_Y, pmpm_Z, biometric_X, biometric_Y):
    common_ids = reduce(np.intersect1d, (pmpm_X.PERS_ID.values, pmpm_Y.PERS_ID.values, pmpm_Z.PERS_ID.values, biometric_X.PERS_ID.values, biometric_Y.PERS_ID.values))
    pmpm_X = pmpm_X.loc[pmpm_X.PERS_ID.isin(common_ids)]
    pmpm_Y = pmpm_Y.loc[pmpm_Y.PERS_ID.isin(common_ids)]
    pmpm_Z = pmpm_Z.loc[pmpm_Z.PERS_ID.isin(common_ids)]
    biometric_X = biometric_X.loc[biometric_X.PERS_ID.isin(common_ids)]
    biometric_Y = biometric_Y.loc[biometric_Y.PERS_ID.isin(common_ids)]
    return pmpm_X, pmpm_Y, pmpm_Z, biometric_X, biometric_Y

In [9]:
def generate_common_patients2(p1, p2, bio):
    common_ids = reduce(np.intersect1d, (p1.PERS_ID.values, p2.PERS_ID.values, bio.PERS_ID.values))
    p1 = p1.loc[p1.PERS_ID.isin(common_ids)]
    p2 = p2.loc[p2.PERS_ID.isin(common_ids)]
    bio = bio.loc[bio.PERS_ID.isin(common_ids)]
    return p1, p2, bio

# Load and Process Data Statistics

In [10]:
biometrics = process_biometrics(biometrics)
#biometrics = pd.read_pickle('./biometrics_processed.pkl')

In [11]:
#biometrics.to_pickle('./biometrics_processed.pkl')

In [12]:
biometrics.head()

BIOMETRIC_CATEGORY  PERS_ID     MYR  A1C  BMI  BNP  \
0                   2561295  200811  0.0  0.0  0.0   
1                   2561295  200912  0.0  0.0  0.0   
2                   2561295  201202  0.0  0.0  0.0   
3                   2561295  201205  0.0  0.0  0.0   
4                   2561295  201209  0.0  0.0  0.0   

BIOMETRIC_CATEGORY  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  \
0                                          0.0                        0.0   
1                                          0.0                        0.0   
2                                          0.0                        0.0   
3                                          0.0                        0.0   
4                                          0.0                        0.0   

BIOMETRIC_CATEGORY  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  ...  \
0                                 0.0        0.0                0.0  ...   
1                                 0.0        0.0                0.0  ...   
2                                 0.0        0.0                0.0  ...   
3                                 0.0        0.0                0.0  ...   
4                                 0.0        0.0                0.0  ...   

BIOMETRIC_CATEGORY  Height  Hematocrit  Hemoglobin  Iron  NT_proBNP  PSA  \
0                      0.0    0.000000        12.5   0.0        0.0  0.0   
1                      0.0    0.000000        12.9   0.0        0.0  0.0   
2                      0.0   34.400002        11.7   0.0        0.0  0.0   
3                      0.0   32.900002        11.2   0.0        0.0  0.0   
4                      0.0   29.600000        10.1   0.0        0.0  0.0   

BIOMETRIC_CATEGORY  Potassium  Troponin  Weight  eGFR  
0                         0.0       0.0     0.0   0.0  
1                         0.0       0.0     0.0   0.0  
2                         0.0       0.0     0.0   0.0  
3                         0.0       0.0     0.0   0.0  
4                         0.0       0.0     0.0   0.0  

[5 rows x 22 columns]

In [13]:
biometrics.MYR.min(), biometrics.MYR.max()

('192001', '217903')

In [14]:
biometrics.MYR = biometrics.MYR.astype(int)
biometric_X = biometrics.loc[(biometrics.MYR>=201801)&(biometrics.MYR<=201812)]
biometric_Y = biometrics.loc[(biometrics.MYR>=201901)&(biometrics.MYR<=201912)]
biometric_X = biometric_X.reset_index(drop=True).sort_values(by=["PERS_ID", "MYR"]).drop(["MYR"], axis=1).groupby(["PERS_ID"]).mean().reset_index()
biometric_Y = biometric_Y.reset_index(drop=True).sort_values(by=["PERS_ID", "MYR"]).drop(["MYR"], axis=1).groupby(["PERS_ID"]).mean().reset_index()

In [15]:
pmpm_X = process_PMPM(pmpm.copy(), 201801, 201812, top_diseases=5, show_range=True)
pmpm_Y = process_PMPM(pmpm.copy(), 201901, 201912, top_diseases=5, show_range=True)
pmpm_Z = process_PMPM(pmpm.copy(), 202001, 202012, top_diseases=5, show_range=True)
#pmpm_X, pmpm_Y, pmpm_Z, biometric_X, biometric_Y = generate_common_patients(pmpm_X, pmpm_Y, pmpm_Z, biometric_X, biometric_Y)
pmpm_X, pmpm_Y0, biometric_X = generate_common_patients2(pmpm_X, pmpm_Y, biometric_X) #train
pmpm_Y1, pmpm_Z, biometric_Y = generate_common_patients2(pmpm_Y, pmpm_Z, biometric_Y) #test

In [16]:
pmpm_X.shape, pmpm_Y0.shape, pmpm_Y1.shape, pmpm_Z.shape, biometric_X.shape, biometric_Y.shape

((144851, 10),
 (144851, 10),
 (167630, 10),
 (167630, 10),
 (144851, 21),
 (167630, 21))

In [17]:
pmpm_Y0

PERS_ID  MIN_MYR  MAX_MYR  MONTHS_MYR  CC_ANXIETY_DISORDER  \
0       2561296   201901   201912          12                    0   
1       2562798   201901   201912          12                    0   
2       2563500   201901   201911          11                    1   
3       2563709   201901   201903           3                    1   
4       2563861   201901   201912          12                    1   
...         ...      ...      ...         ...                  ...   
223306  4401203   201901   201912          12                    0   
223412  5000168   201901   201908           8                    1   
223414  5000189   201901   201912          12                    1   
225132  5010093   201901   201912          12                    0   
227086  5052049   201901   201912          12                    0   

        CC_DEPRESSION  CC_DIABETES  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  \
0                   0            0                  0                0   
1                   0            0                  0                1   
2                   1            0                  0                1   
3                   0            0                  1                0   
4                   1            0                  1                1   
...               ...          ...                ...              ...   
223306              0            0                  1                0   
223412              0            0                  0                0   
223414              0            0                  0                0   
225132              0            0                  0                0   
227086              0            0                  0                0   

        CC_OBESITY  
0                0  
1                0  
2                0  
3                0  
4                0  
...            ...  
223306           0  
223412           0  
223414           0  
225132           0  
227086           0  

[144851 rows x 10 columns]

In [18]:
pmpm_Y1

PERS_ID  MIN_MYR  MAX_MYR  MONTHS_MYR  CC_ANXIETY_DISORDER  \
0       2561296   201901   201912          12                    0   
1       2562798   201901   201912          12                    0   
4       2563861   201901   201912          12                    1   
6       2564177   201901   201912          12                    1   
7       2564249   201901   201912          12                    0   
...         ...      ...      ...         ...                  ...   
227081  5031542   201912   201912           1                    0   
227082  5032099   201912   201912           1                    0   
227083  5032100   201912   201912           1                    0   
227084  5035511   201911   201912           2                    0   
227086  5052049   201901   201912          12                    0   

        CC_DEPRESSION  CC_DIABETES  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  \
0                   0            0                  0                0   
1                   0            0                  0                1   
4                   1            0                  1                1   
6                   1            1                  1                1   
7                   1            0                  0                1   
...               ...          ...                ...              ...   
227081              0            0                  0                0   
227082              0            0                  0                0   
227083              0            0                  0                0   
227084              0            0                  0                0   
227086              0            0                  0                0   

        CC_OBESITY  
0                0  
1                0  
4                0  
6                1  
7                0  
...            ...  
227081           0  
227082           0  
227083           0  
227084           0  
227086           0  

[167630 rows x 10 columns]

In [19]:
pmpm_Z

PERS_ID  MIN_MYR  MAX_MYR  MONTHS_MYR  CC_ANXIETY_DISORDER  \
0       2561296   202001   202006           6                    0   
1       2562798   202001   202011          11                    0   
2       2563861   202001   202011          11                    1   
4       2564177   202001   202011          11                    1   
5       2564249   202001   202006           6                    0   
...         ...      ...      ...         ...                  ...   
228042  5031542   202001   202010          10                    0   
228211  5032099   202001   202011          11                    1   
228212  5032100   202001   202011          11                    0   
229159  5035511   202001   202005           5                    0   
232065  5052049   202001   202011          11                    0   

        CC_DEPRESSION  CC_DIABETES  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  \
0                   0            0                  0                0   
1                   1            0                  0                0   
2                   1            0                  0                1   
4                   1            1                  1                1   
5                   1            0                  0                1   
...               ...          ...                ...              ...   
228042              0            0                  0                0   
228211              1            0                  0                0   
228212              0            0                  0                0   
229159              0            0                  0                0   
232065              0            0                  0                0   

        CC_OBESITY  
0                0  
1                0  
2                0  
4                0  
5                0  
...            ...  
228042           0  
228211           0  
228212           1  
229159           0  
232065           0  

[167630 rows x 10 columns]

In [20]:
biometric_X.head()

BIOMETRIC_CATEGORY  PERS_ID  A1C        BMI        BNP  \
1                   2561296  0.0  27.966973   0.000000   
3                   2562798  0.0  24.945000  20.166667   
5                   2563500  0.0   0.000000   0.000000   
6                   2563709  0.0  10.473750   0.000000   
7                   2563861  0.0   4.080419   0.000000   

BIOMETRIC_CATEGORY  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  \
1                                    86.000000                 120.000000   
3                                    77.444444                 133.333333   
5                                     0.000000                   0.000000   
6                                    45.250000                 100.250000   
7                                    13.600000                  48.400000   

BIOMETRIC_CATEGORY  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  \
1                                 0.0        0.0                0.0   
3                                 0.0        0.0                0.0   
5                                 0.0       55.5                0.0   
6                                 0.0        0.0                0.0   
7                                 0.0        0.0                0.0   

BIOMETRIC_CATEGORY  DCSI-SERUM  ...     Height  Hematocrit  Hemoglobin  \
1                     0.000000  ...  73.000000    0.000000    0.000000   
3                     0.441667  ...  10.416667   19.466667    6.233333   
5                     0.950000  ...   0.000000   45.100000   14.700000   
6                     0.077917  ...  21.750013    4.070833    1.341667   
7                     0.472000  ...  12.200000   29.745714    9.891429   

BIOMETRIC_CATEGORY       Iron  NT_proBNP  PSA  Potassium  Troponin  \
1                    0.000000        0.0  0.0   0.000000       0.0   
3                   12.833333        0.0  0.0   2.166667       0.0   
5                    0.000000        0.0  0.0   4.700000       0.0   
6                    0.000000        0.0  0.0   0.495833       0.0   
7                    0.000000        0.0  0.0   3.085714       0.0   

BIOMETRIC_CATEGORY      Weight     eGFR  
1                   212.000000   0.0000  
3                   138.583333  28.2225  
5                     0.000000  55.5000  
6                    93.475060   0.0000  
7                    21.600000   0.0000  

[5 rows x 21 columns]

In [21]:
biometric_Y.head()

BIOMETRIC_CATEGORY  PERS_ID     A1C        BMI         BNP  \
1                   2561296  0.0000  27.811684    0.000000   
2                   2562798  0.0000  22.407778    0.000000   
5                   2563861  0.0000   6.817982    0.000000   
8                   2564177  3.4375   3.368228    0.000000   
9                   2564249  0.0000   2.191667  322.083333   

BIOMETRIC_CATEGORY  Blood Pressure (Diastolic)  Blood Pressure (Systolic)  \
1                                    70.000000                 108.000000   
2                                    74.222222                 126.111111   
5                                    45.333333                  98.333333   
8                                    18.875000                  59.916667   
9                                     0.000000                  10.833333   

BIOMETRIC_CATEGORY  Cholesterol (LDL)  DCSI-EGFR  DCSI-MICROALBUMIN  \
1                                85.0   0.000000                0.0   
2                                 0.0   5.555556                0.0   
5                                 0.0   0.000000                0.0   
8                                 0.0  19.367857                0.0   
9                                 0.0  47.125000                0.0   

BIOMETRIC_CATEGORY  DCSI-SERUM  ...     Height  Hematocrit  Hemoglobin  Iron  \
1                     1.140000  ...  73.000000   40.099998   14.000000   0.0   
2                     0.740000  ...  34.943739   22.777778    7.244444   0.0   
5                     0.255000  ...  30.333333   12.183333    4.050000   0.0   
8                     0.966179  ...   7.500000    9.709643    3.077500   0.0   
9                     0.980833  ...   0.000000    0.000000    0.000000   0.0   

BIOMETRIC_CATEGORY  NT_proBNP  PSA  Potassium  Troponin      Weight       eGFR  
1                       0.000  0.0   4.100000  0.000000  211.000000   0.000000  
2                       0.000  0.0   3.233333  0.003778  140.652848  27.722222  
5                       0.000  0.0   1.466667  0.000000   35.500000   0.000000  
8                    4398.125  0.0   2.504286  0.010125   35.748396   0.000000  
9                       0.000  0.0   4.212500  0.000000    0.000000  47.125000  

[5 rows x 21 columns]

# Generate Training and Test Data and preprocess

In [22]:
#training data
train_X = pmpm_X.merge(biometric_X, how='left').loc[:,"CC_ANXIETY_DISORDER":"eGFR"]
train_Y = pmpm_Y0.loc[:,"CC_ANXIETY_DISORDER":]

#test data
test_X = pmpm_Y1.merge(biometric_Y, how='left').loc[:,"CC_ANXIETY_DISORDER":"eGFR"]
test_Y = pmpm_Z.loc[:,"CC_ANXIETY_DISORDER":]

In [23]:
#shape
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(144851, 26) (144851, 6) (167630, 26) (167630, 6)


In [24]:
#min_max_scaler = preprocessing.MinMaxScaler((0,1)).fit(train_X)
scaler = preprocessing.StandardScaler().fit(train_X)

/usr/local/lib64/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


# Define Model

In [25]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchsummary import summary
from tqdm import tqdm
from torchvision.transforms import transforms

In [26]:
torch.manual_seed(1)    # reproducible'
x, y = Variable(torch.tensor(scaler.transform(train_X))), Variable(torch.tensor(train_Y.values))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  


In [27]:
model = torch.nn.Sequential(
        torch.nn.Linear(26, 16),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(0.25),
        torch.nn.Linear(16,8),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(0.25),
        torch.nn.Linear(8, 6),
        torch.nn.Sigmoid()
    )
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = torch.nn.BCELoss()
BATCH_SIZE = 128
EPOCH = 200
torch_dataset = Data.TensorDataset(x, y)
loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

In [28]:
summary(model, train_X.shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1           [-1, 144851, 16]             432
         LeakyReLU-2           [-1, 144851, 16]               0
           Dropout-3           [-1, 144851, 16]               0
            Linear-4            [-1, 144851, 8]             136
         LeakyReLU-5            [-1, 144851, 8]               0
           Dropout-6            [-1, 144851, 8]               0
            Linear-7            [-1, 144851, 6]              54
           Sigmoid-8            [-1, 144851, 6]               0
Total params: 622
Trainable params: 622
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 14.37
Forward/backward pass size (MB): 92.83
Params size (MB): 0.00
Estimated Total Size (MB): 107.20
----------------------------------------------------------------


# Training

In [ ]:
for epoch in tqdm(range(EPOCH)):
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x).float()
        b_y = Variable(batch_y).float()
        prediction = model(b_x)
        loss = loss_func(prediction, b_y)
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

 12%|█▎        | 25/200 [01:14<08:41,  2.98s/it]

# Generate Predictions and Ground Truths

In [ ]:
train_prediction = np.round(model(x.float()).detach().numpy())
train_true = y.float().detach().numpy()

In [ ]:
train_prediction

In [ ]:
train_true

In [ ]:
test_x, test_y = Variable(torch.tensor(scaler.transform(test_X))), Variable(torch.tensor(test_Y.values))
test_pred = np.round(model(test_x.float()).detach().numpy())
test_true = test_y.float().detach().numpy()                                                            

In [ ]:
test_pred

In [ ]:
test_true

# Multi-label Evaluation - Subset Accuracy and Hamming Loss

In [33]:
print(f"Multilabel Subset Accuracy for training: {accuracy_score(train_true, train_prediction):0.05f}")
print(f"Multilabel Hamming Loss for training: {hamming_loss(train_true, train_prediction):0.05f}")

Multilabel Subset Accuracy for training: 0.57390
Multilabel Hamming Loss for training: 0.10285


In [34]:
print(f"Multilabel Subset Accuracy for test: {accuracy_score(test_true, test_pred):0.05f}")
print(f"Multilabel Hamming Loss for test: {hamming_loss(test_true, test_pred):0.05f}")

Multilabel Subset Accuracy for test: 0.59460
Multilabel Hamming Loss for test: 0.09720


# Analysis 1 - Persistance Ratio across years

In [37]:
#Disease Code Persistance Rate
#In multilabel classification, this function computes subset accuracy: 
# - the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.
print(accuracy_score(pmpm_X.loc[:,"CC_ANXIETY_DISORDER":], pmpm_Y0.loc[:,"CC_ANXIETY_DISORDER":])) #2018-2019
print(accuracy_score(pmpm_Y1.loc[:,"CC_ANXIETY_DISORDER":], pmpm_Z.loc[:,"CC_ANXIETY_DISORDER":])) #2019-2020

0.6798710398961691
0.7055240708703693


# Analysis 2 - Accuracy in Non-Persistant Patients (state change from 0 to 1) with existing Model

In [40]:
#in 2019
pmpm_Y_0 = pmpm_Y.query("CC_ANXIETY_DISORDER==0 and CC_DEPRESSION==0 and CC_DIABETES==0 and CC_HYPERLIPIDEMIA==0 and CC_HYPERTENSION==0 and CC_OBESITY==0")
#in 2020
pmpm_Z_1 = pmpm_Z.query("CC_ANXIETY_DISORDER==1 or CC_DEPRESSION==1 or CC_DIABETES==1 or CC_HYPERLIPIDEMIA==1 or CC_HYPERTENSION==1 or CC_OBESITY==1")

In [41]:
pmpm_Y_0, pmpm_Z_1, biometric_Y_0 = generate_common_patients2(pmpm_Y_0, pmpm_Z_1, biometric_Y)

In [ ]:
# pmpm_Y_0 = pmpm_Y_0.loc[pmpm_Y_0.PERS_ID.isin(common_ids)]
# pmpm_Z_1 = pmpm_Z_1.loc[pmpm_Z_1.PERS_ID.isin(common_ids)]
# biometric_Y_0 = biometric_Y.loc[biometric_Y.PERS_ID.isin(common_ids)]

In [42]:
test_X = pmpm_Y_0.merge(biometric_Y_0, how='left').loc[:,"CC_ANXIETY_DISORDER":"eGFR"]
test_Y = pmpm_Z_1.loc[:,"CC_ANXIETY_DISORDER":]

In [43]:
test_x, test_y = Variable(torch.tensor(scaler.transform(test_X))), Variable(torch.tensor(test_Y.values))
test_pred = np.round(model(test_x.float()).detach().numpy())
test_true = test_y.float().detach().numpy()                                                            

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [45]:
test_true

array([[0., 0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [46]:
test_pred

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In multilabel classification, the Hamming loss is different from the subset zero-one loss. The zero-one loss considers the entire set of labels for a given sample incorrect if it does not entirely match the true set of labels. Hamming loss is more forgiving in that it penalizes only the individual labels.

In [47]:
print(f"Multilabel Subset Accuracy for test: {accuracy_score(test_true, test_pred):0.05f}")
print(f"Multilabel Hamming Loss for test: {hamming_loss(test_true, test_pred):0.05f}")

Multilabel Subset Accuracy for test: 0.00057
Multilabel Hamming Loss for test: 0.23664


# Analysis 3 - Accuracy in Non-Persistant Patients (state change from 0 to 1) with newly trained Model

Model train will be on (biometrics_X_0, pmpm_X_0) -> pmpm_Y_1 <br/> 
Model test will be on (biometrics_Y_0, pmpm_Y_0) -> pmpm_Z_1 

In [52]:
#in 2018
pmpm_X_0 = pmpm_X.query("CC_ANXIETY_DISORDER==0 and CC_DEPRESSION==0 and CC_DIABETES==0 and CC_HYPERLIPIDEMIA==0 and CC_HYPERTENSION==0 and CC_OBESITY==0")
#in 2019
pmpm_Y_1 = pmpm_Y0.query("CC_ANXIETY_DISORDER==1 or CC_DEPRESSION==1 or CC_DIABETES==1 or CC_HYPERLIPIDEMIA==1 or CC_HYPERTENSION==1 or CC_OBESITY==1")
pmpm_Y_0 = pmpm_Y1.query("CC_ANXIETY_DISORDER==0 and CC_DEPRESSION==0 and CC_DIABETES==0 and CC_HYPERLIPIDEMIA==0 and CC_HYPERTENSION==0 and CC_OBESITY==0")
#in 2020
pmpm_Z_1 = pmpm_Z.query("CC_ANXIETY_DISORDER==1 or CC_DEPRESSION==1 or CC_DIABETES==1 or CC_HYPERLIPIDEMIA==1 or CC_HYPERTENSION==1 or CC_OBESITY==1")

In [53]:
common_ids_XY = np.intersect1d(pmpm_X_0.PERS_ID, pmpm_Y_1.PERS_ID)
common_ids_YZ = np.intersect1d(pmpm_Y_0.PERS_ID, pmpm_Z_1.PERS_ID)

pmpm_X_0 = pmpm_X_0.loc[pmpm_X_0.PERS_ID.isin(common_ids_XY)]
pmpm_Y_1 = pmpm_Y_1.loc[pmpm_Y_1.PERS_ID.isin(common_ids_XY)]

pmpm_Y_0 = pmpm_Y_0.loc[pmpm_Y_0.PERS_ID.isin(common_ids_YZ)]
pmpm_Z_1 = pmpm_Z_1.loc[pmpm_Z_1.PERS_ID.isin(common_ids_YZ)]

biometric_X_0 = biometric_X.loc[biometric_X.PERS_ID.isin(common_ids_XY)]
biometric_Y_0 = biometric_Y.loc[biometric_Y.PERS_ID.isin(common_ids_YZ)]

In [54]:
#training data
train_X = pmpm_X_0.merge(biometric_X_0, how='left').loc[:,"CC_ANXIETY_DISORDER":"eGFR"]
train_Y = pmpm_Y_1.loc[:,"CC_ANXIETY_DISORDER":]

#test data
test_X = pmpm_Y_0.merge(biometric_Y_0, how='left').loc[:,"CC_ANXIETY_DISORDER":"eGFR"]
test_Y = pmpm_Z_1.loc[:,"CC_ANXIETY_DISORDER":]

In [58]:
#shape
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(10229, 26) (10229, 6) (10483, 26) (10483, 6)


In [59]:
torch.manual_seed(1)    # reproducible
x, y = Variable(torch.tensor(scaler.transform(train_X))), Variable(torch.tensor(train_Y.values))
test_x, test_y = Variable(torch.tensor(scaler.transform(test_X))), Variable(torch.tensor(test_Y.values))
model = torch.nn.Sequential(
        torch.nn.Linear(26, 16),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(0.25),
        torch.nn.Linear(16,8),
        torch.nn.LeakyReLU(),
        torch.nn.Dropout(0.25),
        torch.nn.Linear(8, 6),
        torch.nn.Sigmoid()
    )
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = torch.nn.BCELoss()
BATCH_SIZE = 128
EPOCH = 200
torch_dataset = Data.TensorDataset(x, y)
loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
for epoch in tqdm(range(EPOCH)):
    for step, (batch_x, batch_y) in enumerate(loader):
        b_x = Variable(batch_x).float()
        b_y = Variable(batch_y).float()
        prediction = model(b_x)
        loss = loss_func(prediction, b_y)
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

100%|██████████| 200/200 [01:24<00:00,  2.36it/s]


In [61]:
train_prediction = np.round(model(x.float()).detach().numpy())
train_true = y.float().detach().numpy()
test_pred = np.round(model(test_x.float()).detach().numpy())
test_true = test_y.float().detach().numpy()                                                            

In [62]:
print(f"Multilabel Subset Accuracy for training: {accuracy_score(train_true, train_prediction):0.05f}")
print(f"Multilabel Hamming Loss for training: {hamming_loss(train_true, train_prediction):0.05f}")
print(f"Multilabel Subset Accuracy for test: {accuracy_score(test_true, test_pred):0.05f}")
print(f"Multilabel Hamming Loss for test: {hamming_loss(test_true, test_pred):0.05f}")

Multilabel Subset Accuracy for training: 0.11145
Multilabel Hamming Loss for training: 0.22451
Multilabel Subset Accuracy for test: 0.10407
Multilabel Hamming Loss for test: 0.23109
